In [1]:
# this notebook is to utilised transfer learning to train a model on a dataset to classify medical image data on breast cancer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import pandas as pd
import torchvision
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
import matplotlib.pyplot as plt
import os
import skimage.io as io
import time
import copy
import multiprocessing as mp

In [2]:
# get root directory
root_dir = '/media/victoru/B612CEC512CE8A37/ai50/pytorch_test/data/breatcancer_data/archive'
id_dir = os.listdir(root_dir)
label_dir = os.listdir(os.path.join(root_dir,id_dir[0]))
sample_dir = os.listdir(os.path.join(root_dir,id_dir[0],label_dir[0]))
num_classes = len(label_dir)

# create transform for data augmentation: normalize, and convert to tensor
mean = np.array([0.5, 0.5, 0.5])
std = np.array([0.5, 0.5, 0.5])
data_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize([50,50]),
        transforms.Normalize(mean, std)
    ])
# define hyperparameters
batch_size = 100
num_epochs = 10
random_seed = 42
shuffle_dataset = True
validation_split = 0.2

#show image
def imshow(inp, title):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    plt.title(title)
    plt.show()

In [3]:
#create dataset class for breast cancer data

class breastcancerDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        # get root directory
        self.root_dir = root_dir
        # get transform
        self.transform = transform
        # get list of label from directory under root_dir
        id_dir = os.listdir(root_dir)
        label_dir = os.listdir(os.path.join(root_dir,id_dir[0]))
        self.data = pd.DataFrame(columns=['id','label','image_name'])
        
        # loop through id in id_dir and label in label_dir to get image name and label
        for id in id_dir:
            for label in label_dir:
                tempdata = pd.DataFrame(columns=['id','label','image_name'])
                tempdata['image_name'] = os.listdir(os.path.join(root_dir, id, label))
                tempdata['id'] = id
                tempdata['label'] = label
                self.data = pd.concat([self.data, tempdata],ignore_index = True, axis = 0)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # get image name and label from dataframe
        img_name = os.path.join(self.root_dir,self.data.iloc[idx, 0],self.data.iloc[idx, 1],self.data.iloc[idx, 2])
        image = io.imread(img_name)
        label = torch.tensor(int(self.data.iloc[idx, 1]))
                
        if self.transform:
            image = self.transform(image)
            
        return image,label

In [14]:
#check lowest time to load data for given num_workers
lowesttime = 999999999
num_workers_lowest = 0

for num_workers in range (2, mp.cpu_count(),2):
    testloader = DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler, num_workers=num_workers)
    n_total_steps = len(testloader)
    print(n_total_steps)
    start = time.time()
    for epoch in range(1,2):
        for i, data in enumerate(testloader,0):
            if (i+1) % 100 == 0:
                print ('Step [{}/{}]'.format(i+1, n_total_steps))
            pass
    end = time.time()
    percent = (i/n_total_steps)*100
    timetaken = end - start
    print("Finish with: {} second, num_workers={}".format(timetaken,num_workers))
    if timetaken < lowesttime:
        lowesttime = timetaken
        num_workers_lowest = num_workers

print("Lowest time taken: {} second, num_workers={}".format(lowesttime,num_workers_lowest))

556
Step [100/556]
Step [200/556]
Step [300/556]
Step [400/556]
Step [500/556]
Finish with: 18.06663203239441 second, num_workers=2
556
Step [100/556]
Step [200/556]
Step [300/556]
Step [400/556]
Step [500/556]
Finish with: 10.651078939437866 second, num_workers=4
556
Step [100/556]
Step [200/556]
Step [300/556]
Step [400/556]
Step [500/556]
Finish with: 8.501567840576172 second, num_workers=6
556
Step [100/556]
Step [200/556]
Step [300/556]
Step [400/556]
Step [500/556]
Finish with: 7.7006213665008545 second, num_workers=8
556
Step [100/556]
Step [200/556]
Step [300/556]
Step [400/556]
Step [500/556]
Finish with: 7.516961574554443 second, num_workers=10
Lowest time taken: 7.516961574554443 second, num_workers=10


In [ ]:
# create dataset object
dataset = breastcancerDataset(root_dir,data_transforms)

# create data indices for training and validation splits
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# create samplers for training and validation splits
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

# create data loaders
train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler, num_workers=num_workers_lowest)
valid_loader = DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)

In [5]:
#images, classes = next(iter(train_loader))
#print(images.shape[2])
#print(type(images))
#print(images.shape)
#imshow(torchvision.utils.make_grid(images), title=classes)

In [6]:
#setting up device to run on cuda
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
#create training loop function
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in range(2):
            if phase == 0:
                # set model to train mode for training phase
                model.train()
                dataloader = train_loader
            else:
                # set model to evaluate mode for validation phase
                model.eval()
                dataloader = valid_loader
            running_loss = 0.0
            running_corrects = 0

            # Iterate over databatchs for give phase
            n_total_steps = len(dataloader)
            for i, (inputs, labels) in enumerate(dataloader):
                inputs = inputs.to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # enable gradient computation when phase is train and forward pass for both phase
                with torch.set_grad_enabled(phase == 0):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # backward pass only if phase is train
                    if phase == 0:
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                if (i+1) % 10 == 0:
                    print ('Step [{}/{}], Loss: {:.4f}'.format(i+1, n_total_steps, loss.item()))
            epoch_loss = running_loss / dataset_size[phase]
            epoch_acc = running_corrects.double() / dataset_size[phase]
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model and update best_acc if phase is validation and epoch_acc is greater than best_acc
            if phase == 1 and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                print('Best Acc: {:.4f}'.format(best_acc))
            # adjust learning rate using scheduler if phase is validation
            if phase == 1:
                print('Scheduler step')
                scheduler.step()
        print()

    # record time to train model
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

In [8]:
# fine tune model
# define fine tuned model as pretrained resnet34
model_ft = models.resnet18(pretrained=True)

# freeze all existing parameters
for param in model_ft.parameters():
    param.requires_grad = False

#get number of input features of last layer
num_features = model_ft.fc.in_features

#replace last layer with new layer with number of classes as output
model_ft.fc = nn.Linear(num_features, num_classes)

# set model to current device
model_ft = model_ft.to(device)

# define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_ft.parameters(), lr=0.001)

# setup scheduler for learning rate decay
step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [9]:
model_ft = train_model(model_ft, criterion, optimizer, step_lr_scheduler, num_epochs=num_epochs)

Epoch 0/9
----------
Step [10/4441], Loss: 0.7802
Step [20/4441], Loss: 0.6700
Step [30/4441], Loss: 0.7036
Step [40/4441], Loss: 0.5904
Step [50/4441], Loss: 0.4753
Step [60/4441], Loss: 0.5051
Step [70/4441], Loss: 0.6469
Step [80/4441], Loss: 0.6904
Step [90/4441], Loss: 0.5431
Step [100/4441], Loss: 0.4904
Step [110/4441], Loss: 0.5219
Step [120/4441], Loss: 0.4815
Step [130/4441], Loss: 0.5168
Step [140/4441], Loss: 0.4167
Step [150/4441], Loss: 0.7163
Step [160/4441], Loss: 0.4699
Step [170/4441], Loss: 0.5446
Step [180/4441], Loss: 0.5083
Step [190/4441], Loss: 0.4552
Step [200/4441], Loss: 0.3891
Step [210/4441], Loss: 0.5768
Step [220/4441], Loss: 0.3738
Step [230/4441], Loss: 0.3367
Step [240/4441], Loss: 0.3429
Step [250/4441], Loss: 0.3893
Step [260/4441], Loss: 0.3553
Step [270/4441], Loss: 0.6322
Step [280/4441], Loss: 0.3640
Step [290/4441], Loss: 0.4238
Step [300/4441], Loss: 0.4593
Step [310/4441], Loss: 0.5041
Step [320/4441], Loss: 0.5067
Step [330/4441], Loss: 0.553

KeyboardInterrupt: 